In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
1. name — the name of the country.
2. area— the country's total area (both land and water).
3. area_land — the country's land area in square kilometers.
4. area_water — the country's waterarea in square kilometers.
5. population — the country's population.
6. population_growth— the country's population growth as a percentage.
7. birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
8. death_rate — the country's death rate, or the number of death per year per 1,000 people.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT * FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
select *from facts limit 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [4]:
%%sql
SELECT MIN(population), MAX(population),MIN(population_growth),MAX(population_growth) FROM facts  

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [5]:
%%sql 
select name from facts where population =
(select MIN(population) from facts)

Done.


name
Antarctica


In [6]:
%%sql 
select name from facts where population =
(select MAX(population) from facts)

Done.


name
World


In [7]:
%%sql
SELECT MIN(population), MAX(population),MIN(population_growth),MAX(population_growth) FROM facts where population <> (select MIN(population) from facts) AND population <> (select MAX(population) from facts)

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
48,1367485388,0.0,4.02


In [8]:
%%sql
select AVG(population), AVG(area) FROM facts WHERE population <> (select MIN(population) from facts) AND population <> (select MAX(population) from facts) 

Done.


AVG(population),AVG(area)
32377011.0125,582949.8523206752


These are the list of countries where the Population is higher than the average population of a country in the world and area is lower than the average value. 
- *This could show us countries that are overcrowded, lower area and more people*
Let us look at this more deeply.
Let's observe the ratio between population and area. 


In [9]:
%%sql
select name from facts where population > 
(select  AVG(population) from facts where name not in 
 ('Antarctica','World')) AND area < (select AVG(area) from facts 
                                     where name not in 
                                     (select name from facts where population =(select MIN(population) from facts)) )

Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
"Korea, South"
Morocco
Philippines
Poland
Spain


Let us verify if our table above is correct , 

The result above and the result below is the same. We can chose to do it completely dynamically but that would be extremely unreadble, but I have done it for a few fields so illustrate that it is possible.

In [10]:
%%sql
select name from facts where population>32377011.0125 and area <582949.8523206752


Done.


name
Bangladesh
Germany
Iraq
Italy
Japan
Kenya
"Korea, South"
Morocco
Philippines
Poland


Let's observe the ratio between population and area. Let us arrange it in descending order to have a look at the highest to lowest ratio. We see a few common countries between the 2 lists 

*From the earlier analysis we may have naturally assumed that Bangladesh would have been at the top of this list.
However, there are many countries around the world that have a population to area ratio that is higher than Bangldesh. The reason why these countries did not come up is because they have a population thats much less than average world population.*

In [11]:
%%sql
select name from facts order by (population/area) desc limit 25

Done.


name
Macau
Monaco
Singapore
Hong Kong
Gaza Strip
Gibraltar
Bahrain
Maldives
Malta
Bermuda


In [12]:
%%sql
select name, population,population_growth from facts where population = (select MAX(population) from facts where name<>'World' )

Done.


name,population,population_growth
China,1367485388,0.45


In [13]:
%%sql
select name, population_growth from facts where population_growth = (select MAX(population_growth) from facts)

Done.


name,population_growth
South Sudan,4.02


In [14]:
%%sql
select count(*) as 'Countires with above average growth rate' from facts where population_growth > (select AVG(population_growth) from facts)

Done.


Countires with above average growth rate
101


In [15]:
%%sql
select name from facts where population_growth > (select avg(population_growth) from facts ) order by population_growth desc limit 25

Done.


name
South Sudan
Malawi
Burundi
Niger
Uganda
Qatar
Burkina Faso
Mali
Cook Islands
Iraq


In [16]:
%%sql
select name,MAX(cast(area_water as Float)/area_land) from facts

Done.


name,MAX(cast(area_water as Float)/area_land)
British Indian Ocean Territory,905.6666666666666


British Indian Ocean Territory has the highest water:land ratio. 
Let us have a look the lowest water:land ratio, it obviously will not be one country, we know many countries are land locked with no access to water.

In [17]:
%%sql
select name,cast(area_water as Float)/area_land as land_ratio from facts where 
land_ratio =(select MIN(cast(area_water as Float)/area_land) from facts)

Done.


name,land_ratio
Afghanistan,0.0
Algeria,0.0
Andorra,0.0
Angola,0.0
Antigua and Barbuda,0.0
Bahrain,0.0
Barbados,0.0
Bhutan,0.0
Cabo Verde,0.0
Central African Republic,0.0


In [18]:
%%sql
select name, population,population_growth,population*(population_growth/100) as population_increase,population*(population_growth/100)+population as future_population from facts where name !='World' order by population_increase desc

Done.


name,population,population_growth,population_increase,future_population
India,1251695584,1.22,15270686.124799998,1266966270.1248
China,1367485388,0.45,6153684.246,1373639072.246
Nigeria,181562056,2.45,4448270.372,186010326.372
Pakistan,199085847,1.46,2906653.3662,201992500.3662
Ethiopia,99465819,2.89,2874562.1691,102340381.1691
Bangladesh,168957745,1.6,2703323.92,171661068.92
United States,321368864,0.78,2506677.1392,323875541.1392
Indonesia,255993674,0.92,2355141.8008,258348815.8008
"Congo, Democratic Republic of the",79375136,2.45,1944690.8320000002,81319826.832
Philippines,100998376,1.61,1626073.8536,102624449.8536


Ordered the list by highest population increase based on the estimated population growth, now let us order the list by highest population in the future based on our estimate growth

In [19]:
%%sql
select name, population,population_growth,population*(population_growth/100) as population_increase,population*(population_growth/100)+population as future_population from facts where name !='World' order by future_population desc

Done.


name,population,population_growth,population_increase,future_population
China,1367485388,0.45,6153684.246,1373639072.246
India,1251695584,1.22,15270686.124799998,1266966270.1248
European Union,513949445,0.25,1284873.6125,515234318.6125
United States,321368864,0.78,2506677.1392,323875541.1392
Indonesia,255993674,0.92,2355141.8008,258348815.8008
Brazil,204259812,0.77,1572800.5524000002,205832612.5524
Pakistan,199085847,1.46,2906653.3662,201992500.3662
Nigeria,181562056,2.45,4448270.372,186010326.372
Bangladesh,168957745,1.6,2703323.92,171661068.92
Russia,142423773,0.04,56969.5092,142480742.5092


From the 2 lists, it is evident that India and China will top the list for a long time. An interesting thing to note about the **population increase** list is that many African countries and a few Asian countries will see a surge in population.

Let us look at this closesly by observing the birth rate

In [20]:
%%sql
select AVG(birth_rate) from facts where name not in ('World')

Done.


AVG(birth_rate)
19.331762114537433


In [21]:
%%sql
select COUNT(*) as 'Number of countires that have birth rate higher than the average' from facts where birth_rate > (select AVG(birth_rate) from facts where name not in ('World'))

Done.


Number of countires that have birth rate higher than the average
89


In [22]:
%%sql
select name, birth_rate from facts where birth_rate > (select AVG(birth_rate) from facts where name not in ('World')) order by birth_rate desc

Done.


name,birth_rate
Niger,45.45
Mali,44.99
Uganda,43.79
Zambia,42.13
Burkina Faso,42.03
Burundi,42.01
Malawi,41.56
Somalia,40.45
Angola,38.78
Mozambique,38.58


Majoirty of the countires are from Asia and Africa with a birth rate that is above average. We can expect a lot more Asian and African babies than babies from other continents. 

This asserts our above analysis about many Africans and a few Asian countries.

Let us look observe the Death rates

In [23]:
%%sql
select name,round(death_rate-birth_rate,2) from facts where round(death_rate-birth_rate,2) >=0 order by round(death_rate-birth_rate,2) desc

Done.


name,"round(death_rate-birth_rate,2)"
Bulgaria,5.52
Serbia,4.58
Latvia,4.31
Lithuania,4.17
Ukraine,3.74
Hungary,3.57
Germany,2.95
Slovenia,2.95
Romania,2.76
Croatia,2.73


Let us have a look at the list of countries with a higher death rate than birth rate, one thing stands out distinctively is that almost all of these countries are Europen and more specifically Eastern European. 
Perhaps this [study](https://globalriskinsights.com/2019/10/population-decline-in-central-and-eastern-europe/) ought to give us a better understanding.

# Conclusion 

1. African countries are going have an increase in populaion like no other continent
2. Eastern European countries have a negative birth rate(or positive death rate).
3. China and India are clear at the top in terms of population, and India has a higher population growth rate meaning there is a chance India can cross China in terms of population in the future.
4. Population/Area ratio list, asian countires are the majority.

